In [1]:
import pandas as pd
import qgrid
import json
import os
import functools
import ipywidgets as widgets
from pathlib import Path
from IPython.display import clear_output, display
from ipyleaflet import Map, Icon, Marker, AntPath, basemaps, basemap_to_tiles, LayerException
from OSGridConverter import grid2latlong

import sys
sys.path.append('..')
from src import sandbox, gpx
from src.settings import PROJECT_PATH, DATASET_PATH

In [2]:
file_gen = DATASET_PATH.glob("*walks.json")

dfs = [] # an empty list to store the data frames
for file in file_gen:
    
    with open(file) as f:
        
        data = json.load(f)
        dfs.append(pd.DataFrame(data))

df = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.

display(df)

,Distance,Time,Ascent,Start Grid Ref,Area0,Area1,Name,Rating,Votes,Grade,...,Munro,Grahams,Sub 2000s,Corbett / Donald,Graham / Donald,Donald,Corbett:,Donalds,Graham:,Grahams:
0,15.5km / 9.75 miles,3.5 - 4 hours,419m (Profile),NB052077,outer-hebrides,sron-ulladale,Sròn Ulladale stalkers' path,3.00,8,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23.5km / 14.5 miles,6 - 7 hours,494m (Profile),NG154999,outer-hebrides,harris-walkway-2,Harris Walkway 2: Tarbert to Seilebost,3.67,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11.5km / 7.25 miles,NaN,574m (Profile),NG092973,outer-hebrides,beinn-dhubh,"Beinn Dhubh, Losgaintir",4.00,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20.25km / 12.5 miles,NaN,1100m (Profile),NB100062,outer-hebrides,uisgneabhal-mor,"Uisgneabhal Mòr , Teileasbhal and Stuabhal",3.75,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19km / 11.75 miles,6 hours,761m (Profile),NB184004,outer-hebrides,urgha-maraig,Urgha and Maraig circuit,3.80,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809,1.4km / 0.75 miles,40 minutes,31m (Profile),NG789329,kintail,plockton-coral-beach,"Plockton Coral Beach, Plockton",3.89,9,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3810,5.25km / 3.25 miles,2 - 2.5 hours,236m (Profile),NG806284,kintail,Balmacara,Balmacara Woods and Reraig circuit,3.57,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3811,2.5km / 1.5 miles,1 - 1.5 hours,55m (Profile),NG755273,kintail,Plockofkyle,"Plock of Kyle, Kyle of Lochalsh",3.71,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3812,3.25km / 2 miles,1 hour +,43m (Profile),NG803333,kintail,Plockton,Plockton Explorer,4.00,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have two columns for time as some of the higher graded walks are given in summer conditions. Let's combine those.
Similary, some columns are plurals of others, let's combine those too.

In [3]:
df["Time"] = df["Time"].fillna(df["Time (summer conditions)"])
df = df.drop(columns="Time (summer conditions)")

for col in df.columns:
    if col + 's' in df.columns:
        df[col] = df[col].fillna(df[col + 's'])
        df = df.drop(columns=col + 's')

display(df)

,Distance,Time,Ascent,Start Grid Ref,Area0,Area1,Name,Rating,Votes,Grade,...,Sub 2000,Graham,Corbett,Munro,Corbett / Donald,Graham / Donald,Donald,Corbett:,Graham:,Grahams:
0,15.5km / 9.75 miles,3.5 - 4 hours,419m (Profile),NB052077,outer-hebrides,sron-ulladale,Sròn Ulladale stalkers' path,3.00,8,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23.5km / 14.5 miles,6 - 7 hours,494m (Profile),NG154999,outer-hebrides,harris-walkway-2,Harris Walkway 2: Tarbert to Seilebost,3.67,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11.5km / 7.25 miles,4 - 5 hours,574m (Profile),NG092973,outer-hebrides,beinn-dhubh,"Beinn Dhubh, Losgaintir",4.00,7,3,...,Beinn Dhubh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20.25km / 12.5 miles,8 hours,1100m (Profile),NB100062,outer-hebrides,uisgneabhal-mor,"Uisgneabhal Mòr , Teileasbhal and Stuabhal",3.75,4,4,...,Stulabhal,Uisgneabhal Mòr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19km / 11.75 miles,6 hours,761m (Profile),NB184004,outer-hebrides,urgha-maraig,Urgha and Maraig circuit,3.80,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809,1.4km / 0.75 miles,40 minutes,31m (Profile),NG789329,kintail,plockton-coral-beach,"Plockton Coral Beach, Plockton",3.89,9,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3810,5.25km / 3.25 miles,2 - 2.5 hours,236m (Profile),NG806284,kintail,Balmacara,Balmacara Woods and Reraig circuit,3.57,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3811,2.5km / 1.5 miles,1 - 1.5 hours,55m (Profile),NG755273,kintail,Plockofkyle,"Plock of Kyle, Kyle of Lochalsh",3.71,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3812,3.25km / 2 miles,1 hour +,43m (Profile),NG803333,kintail,Plockton,Plockton Explorer,4.00,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's have a look at walks that include Munro's only. While we're at it, let's look at the really tough one's! We'll also drop columns that have all null values.

In [4]:
munro_walks = df[(df['Munro'].notnull()) & (df['Grade'].astype(int) >= 3)]
munro_walks = munro_walks.dropna(axis=1, how='all')
display(munro_walks)

,Distance,Time,Ascent,Start Grid Ref,Area0,Area1,Name,Rating,Votes,Grade,Bog,Link,GPX,Graham,Corbett,Munro
47,8km / 5 miles,5 - 6 hours,990m (Profile),NG560215,skye,blabheinn,Blà Bheinn (Blaven),4.27,45,4,1,https://www.walkhighlands.co.uk/skye/blabheinn...,https://www.walkhighlands.co.uk/skye/profiles/...,NaN,NaN,Blà Bheinn
134,10km / 6.25 miles,4 - 6 hours,1095m (Profile),NG843105,kintail,Beinnsgritheall,Beinn Sgritheall,3.89,9,4,2,https://www.walkhighlands.co.uk/kintail/Beinns...,https://www.walkhighlands.co.uk/kintail/profil...,NaN,NaN,Beinn Sgritheall
192,29km / 18 miles,11 - 14 hours,1826m (Profile),NH218315,lochness,mullardoch-munros,The Loch Mullardoch Munros,3.42,12,4,4,https://www.walkhighlands.co.uk/lochness/mulla...,https://www.walkhighlands.co.uk/lochness/profi...,NaN,NaN,"Càrn nan Gobhar, Sgùrr na Lapaich, An Riabhach..."
193,25km / 15.5 miles,7 - 10 hours,1567m (Profile),NH283386,lochness,strathfarrar-munros,Glen Strathfarrar Munros circuit,3.38,24,4,3,https://www.walkhighlands.co.uk/lochness/strat...,https://www.walkhighlands.co.uk/lochness/profi...,NaN,NaN,"Sgùrr na Ruaidhe, Càrn nan Gobhar, Sgùrr a' Ch..."
198,16.5km / 10.25 miles,6 - 8 hours,1096m (Profile),NH216242,lochness,Tollcreagach,"Tom a' Chòinich and Toll Creagach, Glen Affric",3.65,17,4,3,https://www.walkhighlands.co.uk/lochness/Tollc...,https://www.walkhighlands.co.uk/lochness/profi...,NaN,NaN,"Toll Creagach, Tom a' Chòinich"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749,26km / 16.25 miles,7 - 9 hours,930m (Profile),NN874662,perthshire,carn-a-chlamain,Càrn a'Chlamain,3.65,26,4,2,NaN,https://www.walkhighlands.co.uk/perthshire/pro...,NaN,NaN,Càrn a Chlamain
3761,10km / 6.25 miles,4 - 6 hours,731m (Profile),NN753556,perthshire,schiehallion,Schiehallion,3.36,123,3,1,NaN,https://www.walkhighlands.co.uk/perthshire/pro...,NaN,NaN,Schiehallion
3766,22km / 13.75 miles,8 - 10 hours,1239m (Profile),NN905671,perthshire,beinn-a-ghlo,Beinn a' Ghlò,3.61,66,4,2,NaN,https://www.walkhighlands.co.uk/perthshire/pro...,NaN,NaN,"Càrn Liath, Braigh Coire Chruinn-bhalgain, Càr..."
3771,29km / 18 miles,7 - 9 hours,990m (Profile),NN874662,perthshire,beinn-dearg,Beinn Dearg from near Blair Atholl,3.12,26,4,2,NaN,https://www.walkhighlands.co.uk/perthshire/pro...,NaN,NaN,Beinn Dearg


In [5]:
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(57, 356),
    zoom=6
    )

p = Map(
    basemap=basemap_to_tiles(basemaps.WaymarkedTrails.hiking),
    center=(57, 356),
    zoom=6
    )

#mark = Marker(location=[57, 356], icon=icon)
#m.add_layer(mark)

for p in munro_walks["Start Grid Ref"]:
    loc = grid2latlong(p)
    lat = loc.latitude
    lon = 360 + loc.longitude if loc.longitude < 0 else loc.longitude
    mark = Marker(location=[lat, lon])
    m.add_layer(mark)

m

Map(center=[57, 356], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [9]:
df = sandbox.main()

widget = qgrid.show_grid(df)
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [7]:
def get_lat_lon_bounds(df):
    
    north = max(df["lat"])
    south = min(df["lat"])
    east = max(df["lon"])
    west = min(df["lon"])
    
    return [(south, west), (north, east)]

def handle_click(map_handle, data, **kwargs):
    
    route = gpx.parse(data["GPX"])
    route = gpx.positive_long(route)
    latlon = list(zip(route.lat, route.lon))
    path = AntPath(locations=latlon)
    
    try:
        map_handle.substitute_layer(handle_click.current, path)
    except (AttributeError, LayerException):
        map_handle.add_layer(path)
        
    handle_click.current = path
    
    map_handle.center = kwargs["coordinates"]
    map_handle.zoom = 11
    # map_handle.fit_bounds = get_lat_lon_bounds(route)
   
    display(data)
    

In [8]:
filtered_df = widget.get_changed_df()

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(57, 356),
    zoom=6
    )

display_max = 100

if len(filtered_df.index) < display_max:
    
    for _, p in filtered_df.iterrows():
        mark = Marker(location=(p["lat"], p["lon"]), draggable=False, title=p["Name"])
        mark.on_click(functools.partial(handle_click, m, p))
        m.add_layer(mark)
        
else:
    print(f"Too many walks to show ({len(filtered_df.index)})! Filter below maximum ({display_max}) ")
    
m
        

Too many walks to show (3814)! Filter below maximum (100) 


Map(center=[57, 356], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…